In [21]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [22]:
import NN_models
import Filters
import utils
import Systems
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateKalmanNet import StateKalmanNet

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [24]:
h_true_nonlinear = lambda x: 0.5 * x
f_true_nonlinear = lambda x: 0.9 * x - 0.05 * x**3 

Q_true = torch.tensor([[0.1]])
R_true = torch.tensor([[0.1]])

Ex0_true = torch.tensor([[1.0]])
P0_true = torch.tensor([[0.5]])

sys_true = Systems.NonlinearSystem(f_true_nonlinear, h_true_nonlinear, Q_true, R_true, Ex0_true, P0_true)

#  Nepřesná dynamika (lineární aproximace nelineární funkce f)
f_model_nonlinear = lambda x: 0.9 * x 
h_model_nonlinear = h_true_nonlinear
# Nepřesná znalost šumu (podcenění Q)
Q_model = torch.tensor([[0.01]])
R_model = torch.tensor([[0.2]])
# Nepřesný počáteční odhad (pro EKF)
Ex0_model = torch.tensor([[0.5]])
P0_model = torch.tensor([[0.5]])

# Sestavení nepřesného modelu pro filtry
# Funkce h, R jsou pro jednoduchost stejné, ale f, Q, Ex0, P0 jsou jiné
sys_model = Systems.NonlinearSystem(f_model_nonlinear, h_model_nonlinear, Q_model, R_model, Ex0_model, P0_model)
# sys_model = Systems.NonlinearSystem(f_true_nonlinear, h_true_nonlinear, Q_true, R_true, Ex0_model, P0_model)

In [30]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 200      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 5000   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 500    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 32         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [26]:
ekf = Filters.ExtendedKalmanFilter(sys_model)
y_test_seq = y_test.squeeze(0)
ekf_results = ekf.apply_filter(y_test_seq)
x_hat_ekf = ekf_results['x_filtered']
P_hat_ekf = ekf_results['P_filtered']

In [27]:
state_knet = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=10).to(device)
utils.train_state_KalmanNet(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=100, 
    lr=1e-4,
    early_stopping_patience=15
)

Epoch [5/100], Train Loss: 0.200713, Val Loss: 0.173253
Epoch [10/100], Train Loss: 0.165150, Val Loss: 0.150786
Epoch [15/100], Train Loss: 0.154809, Val Loss: 0.145059
Epoch [20/100], Train Loss: 0.154092, Val Loss: 0.144638
Epoch [25/100], Train Loss: 0.153813, Val Loss: 0.144495
Epoch [30/100], Train Loss: 0.153804, Val Loss: 0.144356
Epoch [35/100], Train Loss: 0.153535, Val Loss: 0.144279
Epoch [40/100], Train Loss: 0.153767, Val Loss: 0.144270
Epoch [45/100], Train Loss: 0.153470, Val Loss: 0.144100
Epoch [50/100], Train Loss: 0.153278, Val Loss: 0.144032
Epoch [55/100], Train Loss: 0.153144, Val Loss: 0.143987
Epoch [60/100], Train Loss: 0.153506, Val Loss: 0.143958
Epoch [65/100], Train Loss: 0.153182, Val Loss: 0.143913
Epoch [70/100], Train Loss: 0.153171, Val Loss: 0.143926
Epoch [75/100], Train Loss: 0.153627, Val Loss: 0.143862
Epoch [80/100], Train Loss: 0.153002, Val Loss: 0.143832
Epoch [85/100], Train Loss: 0.152890, Val Loss: 0.143836
Epoch [90/100], Train Loss: 0.15

StateKalmanNet(
  (dnn): DNN_KalmanNet(
    (input_layer): Linear(in_features=2, out_features=20, bias=True)
    (gru): GRU(20, 20)
    (output_layer): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [31]:
import torch
import torch.nn.functional as F
import numpy as np

state_knet.eval()

# Seznamy pro sběr MSE z každé trajektorie
all_mse_knet = []
all_mse_ekf = []

print(f"Vyhodnocuji na {NUM_TEST_TRAJ} testovacích trajektoriích...")

with torch.no_grad():
    # Smyčka přes všechny testovací trajektorie
    for i in range(NUM_TEST_TRAJ):
        # Získáme i-tou trajektorii
        y_test_seq_gpu = y_test[i].to(device) # Tvar [seq_len, obs_dim] pro KNet
        x_true_seq_cpu = x_test[i].cpu()   # Tvar [seq_len, state_dim] pro porovnání
        
        # --- Vyhodnocení StatefulKalmanNet ---
        state_knet.reset(batch_size=1)
        knet_predictions = []
        for t in range(TEST_SEQ_LEN):
            y_t = y_test_seq_gpu[t, :].unsqueeze(0)
            x_filtered_t = state_knet.step(y_t)
            knet_predictions.append(x_filtered_t.squeeze(0))
        x_hat_knet_gpu = torch.stack(knet_predictions, dim=0)
        
        # Spočítáme MSE pro tuto jednu trajektorii a uložíme
        mse_knet_run = F.mse_loss(x_hat_knet_gpu.cpu(), x_true_seq_cpu)
        all_mse_knet.append(mse_knet_run.item())
        
        # --- Vyhodnocení ExtendedKalmanFilter ---
        
        # Vytvoříme novou, čistou instanci EKF pro každou trajektorii
        ekf_instance = Filters.ExtendedKalmanFilter(sys_model)
        
        # Připravíme data - EKF je také očekává na správném zařízení
        y_test_seq_ekf = y_test[i].to(ekf_instance.device)
        
        # Zavoláme metodu .apply_filter
        ekf_results = ekf_instance.apply_filter(y_test_seq_ekf)
        
        # Získáme odhady a přesuneme je na CPU pro porovnání
        x_hat_ekf_seq = ekf_results['x_filtered'].cpu()
        
        # Spočítáme MSE pro tuto jednu trajektorii a uložíme
        mse_ekf_run = F.mse_loss(x_hat_ekf_seq, x_true_seq_cpu)
        all_mse_ekf.append(mse_ekf_run.item())

# --- Finální výpočet průměrného MSE ---
avg_mse_knet = np.mean(all_mse_knet)
avg_mse_ekf = np.mean(all_mse_ekf)

print(f"\nPrůměrná MSE KalmanNet na {NUM_TEST_TRAJ} trajektoriích: {avg_mse_knet:.4f}")
print(f"Průměrná MSE EKF na {NUM_TEST_TRAJ} trajektoriích:       {avg_mse_ekf:.4f}")

Vyhodnocuji na 100 testovacích trajektoriích...

Průměrná MSE KalmanNet na 100 trajektoriích: 0.1339
Průměrná MSE EKF na 100 trajektoriích:       0.2488


In [ ]:
import torch
import matplotlib.pyplot as plt
import time
import numpy as np

# --- 1. Příprava ---

# Předpoklady (musí být definováno v předchozích buňkách)
# state_knet = ... (váš natrénovaný model StatefulKalmanNet)
# sys_true = ...
# device = ...
# TEST_SEQ_LEN = 200

print("Připravuji data pro online simulaci...")
# Vygenerujeme jednu novou, dlouhou trajektorii pro demonstraci
x_online_test, y_online_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

# Přesuneme data na správná zařízení a upravíme tvary
y_online_seq = y_online_test.squeeze(0).to(device) # Tvar [seq_len, obs_dim]
x_true_seq = x_online_test.squeeze(0).cpu()      # Tvar [seq_len, state_dim]

# --- 2. Simulace online filtrace ---

print("Zahajuji simulaci online filtrace...")

# Ujistíme se, že je model v evaluačním módu
state_knet.eval()

# DŮLEŽITÉ: Resetujeme vnitřní stav filtru na začátku nové "mise"
state_knet.reset(batch_size=1)

# Seznam pro sběr online odhadů
online_predictions = []
# Seznam pro měření času na jeden krok
step_times = []

# Vypneme počítání gradientů pro maximální rychlost
with torch.no_grad():
    # Smyčka simulující přicházející data v reálném čase
    for t in range(TEST_SEQ_LEN):
        
        # Změříme čas začátku
        start_time = time.time()
        
        # Získáme jedno nové měření
        y_t = y_online_seq[t, :].unsqueeze(0) # Tvar [1, obs_dim]
        
        # Provedeme JEDEN krok filtrace
        x_filtered_t = state_knet.step(y_t)
        
        # Změříme čas konce a uložíme
        end_time = time.time()
        step_times.append(end_time - start_time)
        
        # Uložíme si odhad (přesunutý na CPU a bez batch dimenze)
        online_predictions.append(x_filtered_t.cpu().squeeze(0))
        
        # (Volitelné) Simulace "čekání" na další měření
        # time.sleep(0.01) 

print("Online simulace dokončena.")

# --- 3. Zpracování a vyhodnocení výsledků ---

# Spojíme odhady z jednotlivých kroků do jedné trajektorie
x_hat_online = torch.stack(online_predictions, dim=0)

# Vypočítáme celkové MSE
mse_online = F.mse_loss(x_hat_online, x_true_seq)
avg_step_time_ms = np.mean(step_times) * 1000 # Průměrný čas v milisekundách

print("\n--- Výsledky online simulace ---")
print(f"  Celková MSE: {mse_online.item():.4f}")
print(f"  Průměrný čas na jeden krok filtrace: {avg_step_time_ms:.4f} ms")

# --- 4. Vizualizace ---

plt.figure(figsize=(18, 9))
plt.title("Simulace online predikce pomocí stavového KalmanNetu", fontsize=16)

# Převod na numpy pro plotování
x_true_plot = x_true_seq.numpy()
y_meas_plot = y_online_seq.cpu().numpy()
x_hat_online_plot = x_hat_online.numpy()

plt.plot(x_true_plot, 'k-', linewidth=3, label="Skutečný stav (Ground Truth)")
plt.plot(y_meas_plot, 'r.', markersize=6, alpha=0.6, label="Přicházející měření")
plt.plot(x_hat_online_plot, 'c--', linewidth=2.5, label=f"Online odhad KalmanNet (MSE={mse_online.item():.4f})")

plt.xlabel("Časový krok (t)", fontsize=12)
plt.ylabel("Hodnota", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.show()